# <h><center>⭐️⭐️Scrabble Player Rating(Predict players' ratings based on Woogles.io gameplay)⭐️⭐️</center></h>
<img src='https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn.vox-cdn.com%2Fthumbor%2F68c3cH43YrqhhjJOWczTlra59a8%3D%2F0x34%3A640x394%2F1600x900%2Fcdn.vox-cdn.com%2Fuploads%2Fchorus_image%2Fimage%2F29923105%2F937649420_d471237362_z.0.jpg&f=1&nofb=1&ipt=16351708cb943c73ec8fc044fe9657fc25b97caa638d15e744f0753a5bfb1c7f&ipo=images'>

## ***This Notebook is ordinary baseline approach only train, test ,sample csv file used and applied preprocessing, Build the XGBoost regression method.***

## **⭐️Try it yourself guys, simple baseline approach⭐️***

# **Import Necessary library**

In [ ]:
#Import necessay libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
import seaborn as sns

#Preprocessing
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


# **Load Data file**

In [ ]:
#import the data and shape
train = pd.read_csv("../input/scrabble-player-rating/train.csv")
test = pd.read_csv("../input/scrabble-player-rating/test.csv")
sample = pd.read_csv("../input/scrabble-player-rating/sample_submission.csv")
games = pd.read_csv("../input/scrabble-player-rating/games.csv")
turns = pd.read_csv("../input/scrabble-player-rating/turns.csv")

print(train.shape,test.shape,sample.shape,games.shape,turns.shape)


In [ ]:
display(train.head())
display(test.head())
display(sample.head())
display(games.head())
display(turns.head())

# **KFold**

In [ ]:
#add extra one columns
train['kfold']=-1
#Distributing the data 5 shares
kfold = model_selection.KFold(n_splits=5, shuffle= True, random_state = 42)
for fold, (train_indicies, valid_indicies) in enumerate(kfold.split(X=train)):
    #print(fold,train_indicies,valid_indicies)
    train.loc[valid_indicies,'kfold'] = fold

    
print(train.kfold.value_counts()) #total data 300000 = kfold split :5 * 60000

#output of train folds data
train.to_csv("trainfold_5.csv",index=False)

In [ ]:
train = pd.read_csv('./trainfold_5.csv')

# **Preprocessing**

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test = test.drop('rating',axis=1)

In [ ]:
train['nickname'] =train['nickname'].astype('category').cat.codes

test['nickname'] =test['nickname'].astype('category').cat.codes

# **Feature spliting**

In [ ]:
X = train.drop(['rating','kfold'],axis=1)
y = train['rating']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

# **Build the Model**

In [ ]:
#store the final_prediction data and score
final_predictions = []
score= []


for fold in range(5):
  
    #Model hyperparameter of XGboostRegressor
    xgb_params = {
        'learning_rate': 0.0011896,
        'subsample': 0.7875490025178,
        'colsample_bytree': 0.11807135201147,
        'max_depth': 6,
        'booster': 'gbtree', 
        'reg_lambda': 0.0008746338866473539,
        'reg_alpha': 23.13181079976304,
        'random_state':40,
        'n_estimators':30000
        
        
    }
    
    model= XGBRegressor(**xgb_params,
                       tree_method='gpu_hist',
                       predictor='gpu_predictor',
                       gpu_id=0)
    model.fit(x_train,y_train,early_stopping_rounds=300,eval_set=[(x_test,y_test)],verbose=2000)
    preds_valid = model.predict(x_test)
    
    #Training model apply the test data and predict the output
    test_pre = model.predict(test)
    final_predictions.append(test_pre)
    
    #Rootmeansquared output
    rms = mean_squared_error(y_test,preds_valid,squared=False)
    
    score.append(rms)
    #way of output is display
    print(f"fold:{fold},rmse:{rms}")

#mean of repeation of fold data and identify the  mean and standard deviation 
print(np.mean(score),np.std(score))


# **Prediction and generate submission csv file**

In [ ]:
#prediction of data
preds = np.mean(np.column_stack(final_predictions),axis=1)
print(preds)
sample.rating = preds[0:22363]
sample.to_csv("submission.csv",index=False)
print("success")

## **Thanks for visiting guys**